In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
import time
import datetime
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg, Robot
# from Jetbot_Servo_Lib import Servo_Serial_Ctrl
from SCSCtrl import TTLServo

Succeeded to open the port
Succeeded to change the baudrate


In [2]:
# 1. 모델 로드 (제공해주신 코드 반영)
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy_test.pth'))
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half() # 연산 속도 최적화

In [3]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

In [4]:
def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
# 3. 색상 시퀀스 및 HSV 설정
# 사용자가 제공한 HSV 예시를 바탕으로 범위를 설정했습니다.
color_ranges = {
    'green':  ([40, 30, 50], [80, 255, 255]),
    'purple': ([110, 40, 40], [160, 255, 255]),
    'blue':   ([100, 100, 50], [130, 255, 255]),
    'red':    ([0, 100, 100], [10, 255, 255]), # 또는 [170, 100, 100]~[180, 255, 255]
    'yellow': ([0, 30, 140], [35, 100, 255])
    #'orange': ([10, 90, 150], [30, 150, 200])
}

# 탐색 순서: 초록 -> 보라 -> 파랑 -> 빨강 -> (다시) 노랑(종료)
target_sequence = ['green', 'purple', 'blue', 'red', 'yellow']
current_step = 0
is_scanning = False
road_running = False
stop_on_green = True 
pending_drop = False  
is_dropping = False 

In [6]:
camera = Camera()
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [7]:
# TTLServo = Servo_Serial_Ctrl()
robot = Robot()

In [8]:
TTLServo.servoAngleCtrl(5, 50, 1, 100)

748

In [9]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [11]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [12]:
# 5. 주요 기능 함수
def start_road_following(drop_on_green=True):
    global road_running, pending_drop
    if road_running:
        return
    road_running = True
    pending_drop = drop_on_green
    camera.observe(execute, names='value')
    print("[ROAD] started")

def stop_road_following():
    global road_running
    if not road_running:
        return
    road_running = False
    try:
        camera.unobserve(execute, names='value')
    except Exception:
        pass
    robot.stop()
    print("[ROAD] stopped")
    
def look_right():
    """노드 탐지 시 동작: 카메라 들기 -> 우측 회전 -> 사진 저장 -> 복귀"""
    global is_scanning
    is_scanning = True
    robot.stop()
    print(f"Node [{target_sequence[current_step]}] Detected. Scanning...")

    # 사진 촬영 및 저장
    now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"node_{target_sequence[current_step]}_{now}.jpg"
    # 현재 화면 저장 (OpenCV는 BGR 사용)
    cv2.imwrite(filename, cv2.cvtColor(camera.value, cv2.COLOR_RGB2BGR))
    print(f"Photo saved: {filename}")

    # 카메라 각도 조절 (부딪힘 방지)
    TTLServo.servoAngleCtrl(5, 1, 1, 150) # Y축 1도로 들기
    time.sleep(4.0)
    TTLServo.servoAngleCtrl(1, 80, 1, 150) # X축 80도 우측 회전
    time.sleep(5.0) 

    # 원위치 복귀
    TTLServo.servoAngleCtrl(5, 1, 1, 150) 
    time.sleep(3.0)
    TTLServo.servoAngleCtrl(1, 0, 1, 150) # 정면
    time.sleep(4.0)
    TTLServo.servoAngleCtrl(5, 50, 1, 100) # 주행 각도(50도)로 복귀
    time.sleep(3.0)

    is_scanning = False
    

In [13]:
def detect_color(image, color_name):

    """특정 영역(ROI)에서 색상 존재 여부 확인"""

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower = np.array(color_ranges[color_name][0])

    upper = np.array(color_ranges[color_name][1])

    

    # ROI 설정 (하단 1/4 영역)

    roi = hsv[160:220, 50:170]

    mask = cv2.inRange(roi, lower, upper)

    mask_ratio = np.sum(mask > 0) / mask.size

    return mask_ratio > 0.40 # 15% 이상 채워지면 감지로 판단

In [14]:
# 6. 메인 실행 함수
angle = 0.0
angle_last = 0.0
frame_count = 0

def execute(change):
    global angle, angle_last, is_scanning, current_step, frame_count
    global road_running, pending_drop, is_dropping
    
    if not road_running:
        return
    
    if is_scanning or is_dropping:
        return
    
    image = change['new']
    frame_count += 1

    # [A] 색상 감지 로직 (3프레임에 한 번)
    if frame_count % 3 == 0:
        target_color = target_sequence[current_step]
        if detect_color(image, target_color):

            # ✅ green이면: stop -> (예약되어 있으면) drop 1회 실행
            if target_color == 'green' and stop_on_green:
                print("Green Node Detected. Stop road following.")
                stop_road_following()

                # 여기서 딱 1번만 내려놓기
                if pending_drop and (not is_dropping):
                    is_dropping = True
                    pending_drop = False  # 중복 방지

                    sleep(0.3)  # 멈춘 직후 안정화
                    goto_pose_loose(POSE_B, move_time=1.5, loose_delay=3.0, loose_angle=20)


                    # (선택) 홈 복귀
                    return_home_keep_grip(homeX=130, homeY=50, pan_home=0, tilt_home=0, move_time=3.0)

                    is_dropping = False
                    print("[DROP] done")

                return
            
            # ✅ yellow면 그냥 종료
            if target_color == 'yellow':
                print("Final Yellow Node Reached. Stopping...")
                stop_road_following()
                return
            
            # ✅ 그 외 노드 처리
            look_right()
            current_step += 1
            frame_count = 0
            return

    # [B] 라인 트래킹 로직
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

    # image_widget.value = bgr8_to_jpeg(image)


In [15]:
from time import sleep

# =========================
# 1) 목표 포즈(B)
# =========================
POSE_B = {
    "goalX": 179,
    "goalY": -60,
    "goalPan": 44,
    "gripAngle": -60
}

# =========================
# 목표 포즈로 이동 후, 지연 뒤 그리퍼 "loose(벌리기)"
# =========================
def goto_pose_loose(pose,
                    move_time=1.5,
                    pan_speed=200,
                    loose_speed=150,
                    loose_delay=4.0,
                    loose_angle=20):
    """
    - pose 위치로 이동
    - 마지막에 그리퍼를 '잡기'가 아니라 '벌리기(loose)'로만 동작
    """
    global goalX, goalY, goalPan, gripAngle

    # 1) 팬(서보 1)
    goalPan = pose["goalPan"]
    TTLServo.servoAngleCtrl(1, goalPan, 1, pan_speed)
    sleep(0.05)

    # 2) 팔 XY 이동
    goalX = pose["goalX"]
    goalY = pose["goalY"]
    TTLServo.xyInputSmooth(goalX, goalY, move_time)

    # 이동 안정화 + 추가 대기
    sleep(move_time + loose_delay)

    # 3) 그리퍼(서보 4) loose(벌리기)
    gripAngle = loose_angle
    TTLServo.servoAngleCtrl(4, gripAngle, 1, loose_speed)
    sleep(2)


# =========================
# 2) 목표 포즈로 이동 후, 지연 뒤 그립
# =========================
def goto_pose(pose, move_time=1.5, pan_speed=200, grip_speed=150, grip_delay=4.0):
    global goalX, goalY, goalPan, gripAngle

    # 1) 팬(서보 1) 먼저 맞추기
    goalPan = pose["goalPan"]
    TTLServo.servoAngleCtrl(1, goalPan, 1, pan_speed)
    sleep(0.05)

    # 2) 팔 XY 먼저 이동
    goalX = pose["goalX"]
    goalY = pose["goalY"]
    TTLServo.xyInputSmooth(goalX, goalY, move_time)

    # 이동 안정화 + 추가 대기
    sleep(move_time + grip_delay)

    # 3) 그리퍼(서보 4) 나중에 잡기
    gripAngle = pose["gripAngle"]
    TTLServo.servoAngleCtrl(4, gripAngle, 1, grip_speed)
    sleep(2)

# =========================
# 3) 홈으로 복귀 (그립 유지!!)
# =========================
def return_home_keep_grip(homeX=130, homeY=50,
                          pan_home=0, tilt_home=0,
                          move_time=3.0, pan_speed=100, tilt_speed=100):
    global goalX, goalY, goalPan, goalC

    # ⚠️ 그리퍼(서보 4)는 절대 건드리지 않음 (물체 유지)

    # 1) 팬(서보 1) 홈
    goalPan = pan_home
    TTLServo.servoAngleCtrl(1, goalPan, 1, pan_speed)
    sleep(0.05)

    # 2) 카메라 틸트(서보 5) 홈
    goalC = tilt_home
    #TTLServo.servoAngleCtrl(5, goalC, 1, tilt_speed)
    sleep(0.05)

    # 3) 팔 XY 홈
    goalX = homeX
    goalY = homeY
    TTLServo.xyInputSmooth(goalX, goalY, move_time)
    sleep(move_time + 0.3)

# =========================
# 4) 실행 예시: B로 이동 → (3초 뒤) 그립 → 홈으로 복귀(그립 유지)
# =========================
goto_pose(POSE_B, move_time=1.5, grip_delay=3.0)
return_home_keep_grip(homeX=130, homeY=50, pan_home=0, tilt_home=0, move_time=3.0)

pending_drop = True
is_dropping = False

start_road_following()
#execute({'new': camera.value})

#goto_pose_loose(POSE_B, move_time=1.5, loose_delay=3.0, loose_angle=20)
#return_home_keep_grip(homeX=130, homeY=50, pan_home=0, tilt_home=0, move_time=3.0) 
# 초기 주행 각도 세팅
# time.sleep(1.0)

[ROAD] started
Green Node Detected. Stop road following.
[ROAD] stopped
[DROP] done


In [16]:
#camera.observe(execute, names='value')

In [17]:
#import time
#camera.unobserve(execute, names='value')
#time.sleep(0.1) 
#robot.stop()
#camera.stop()